In [1]:
from utils.spark_util import read_config, AzureConnector, AsureReader

In [2]:
source = "abfss://m07sparksql@bd201stacc.dfs.core.windows.net/"
                

In [3]:
config = read_config("config.yml")['azure']
session = AzureConnector(config=config).get_session()

22/04/19 15:38:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/19 15:38:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.sql.functions import isnan, when, count, col, concat, lit, udf, when, array, coalesce, min, max, desc, abs

In [5]:
hotels_weather = AsureReader(session = session, data_path = f"{source}hotel-weather", file_format = "parquet").read()

22/04/19 15:38:19 WARN SSLSocketFactoryEx: Failed to load OpenSSL. Falling back to the JSSE default.


In [6]:
hotels_weather.show(10)

+--------------------+----------+----------+------------+-------+-------+-------------+----------+-----------+--------------------+----------+----+-----+---+
|             address|avg_tmpr_c|avg_tmpr_f|        city|country|geoHash|           id|  latitude|  longitude|                name| wthr_date|year|month|day|
+--------------------+----------+----------+------------+-------+-------+-------------+----------+-----------+--------------------+----------+----+-----+---+
|           La Quinta|      27.3|      81.2|    Beaumont|     US|   9vm4| 412316860416|  30.03867|  -94.15933|      5820 Walden Rd|2017-08-06|2017|    8|  6|
|              Ramada|      25.4|      77.8|      Laurel|     US|   dj8s| 455266533377| 31.693611|  -89.13289|     1105 Sawmill Rd|2017-08-06|2017|    8|  6|
|Days Inn-amarillo...|      22.4|      72.3|    Amarillo|     US|   9wr8|1451698946052| 35.188787|-101.920465|   2102 S Coulter St|2017-08-06|2017|    8|  6|
|Knights Inn Amari...|      22.4|      72.3|    Amar

In [7]:
expedia = AsureReader(session = session, data_path = f"{source}expedia", file_format = "avro").read()

In [8]:
expedia.show(10)

+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+
| id|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|     hotel_id|
+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+
|  4|2015-07-17 09:32:04|        2|             3|                   66|                 46

In [9]:
expedia.printSchema()

root
 |-- id: long (nullable = true)
 |-- date_time: string (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: string (nullable = true)
 |-- srch_co: string (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- hotel_id: long (nullable = true)



TOP 10 HOTELS WITH MAXIMAL MONTHLY TEMPERATURE DIFFERENCE

In [ ]:
hotels_weather.groupBy("id", "name", "year", "month").agg(abs(max("avg_tmpr_c")-min("avg_tmpr_c")).alias("monthly_diff")).orderBy(desc("monthly_diff")).show()

+-------------+--------------------+----+-----+------------------+
|           id|                name|year|month|      monthly_diff|
+-------------+--------------------+----+-----+------------------+
|1571958030336|       316 N Main St|2017|    9|              19.1|
| 146028888072|  2799 W Airport Way|2017|    9|              18.5|
|1571958030336|       316 N Main St|2016|   10|              18.5|
|  77309411328|         Po Box 1327|2016|   10|              18.2|
| 695784701957|     2011 Rodgers Dr|2016|   10|              16.3|
| 369367187456|555 E Lafayette Blvd|2016|   10|16.200000000000003|
| 146028888068|     501 National St|2017|    9|15.500000000000002|
|  25769803779| 663 E Cloverland Dr|2017|    9|              15.1|
|  77309411335|    1400 Historic Dr|2016|   10|14.899999999999999|
|1520418422787|  2 Little Beaver Rd|2016|   10|14.899999999999999|
|1013612281860|     485 Hamilton St|2016|   10|              14.7|
| 137438953473|   3250 Northview Dr|2016|   10|14.400000000000

Top 10 busy (e.g., with the biggest visits count) hotels for each month

In [ ]:
min_checkin = expedia.select(min("srch_ci")).show()
min_checkin = expedia.select(max("srch_co")).show()

+------------+
|min(srch_ci)|
+------------+
|  2016-10-01|
+------------+

+------------+
|max(srch_co)|
+------------+
|  2018-10-06|
+------------+



In [ ]:
from pyspark.sql.functions import year, month, to_date, trunc, months_between

In [ ]:
expedia.withColumn("srch_ci_year", year(to_date(col("srch_ci")))).show()

+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+------------+
| id|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|     hotel_id|srch_ci_year|
+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+------------+
|  4|2015-07-17 09:32:04|        2|             3|  

In [ ]:
expedia_with_months = (expedia
                            .withColumn("srch_ci_month", trunc(to_date(col("srch_ci")), "month"))
                            .withColumn("srch_co_month", trunc(to_date(col("srch_co")), "month"))
                            .withColumn("monthDiff",  months_between(col("srch_co_month"), col("srch_ci_month")).cast("Int")))
expedia_with_months.persist()
expedia_with_months.show(5)

+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+-------------+-------------+---------+
| id|          date_time|site_name|posa_continent|user_location_country|user_location_region|user_location_city|orig_destination_distance|user_id|is_mobile|is_package|channel|   srch_ci|   srch_co|srch_adults_cnt|srch_children_cnt|srch_rm_cnt|srch_destination_id|srch_destination_type_id|     hotel_id|srch_ci_month|srch_co_month|monthDiff|
+---+-------------------+---------+--------------+---------------------+--------------------+------------------+-------------------------+-------+---------+----------+-------+----------+----------+---------------+-----------------+-----------+-------------------+------------------------+-------------+-------------+--

In [ ]:
print(expedia_with_months.filter(expedia_with_months["monthDiff"]>0).count())
print(expedia_with_months.filter(expedia_with_months["monthDiff"]>1).count())

189111
316


In [ ]:
expedia_with_months.filter(expedia_with_months["monthDiff"]==0).groupBy("hotel_id", "srch_ci_month").agg(count("id").alias("count_search_requests")).show(10)

+-------------+-------------+---------------------+
|     hotel_id|srch_ci_month|count_search_requests|
+-------------+-------------+---------------------+
| 627065225216|   2016-10-01|                  335|
|1941325217799|   2017-09-01|                  312|
|1391569403908|   2017-09-01|                  300|
|2568390443012|   2016-10-01|                  312|
|2817498546177|   2016-10-01|                  282|
|2980707303434|   2017-08-01|                  302|
|2044404432898|   2016-10-01|                  353|
|2869038153732|   2017-09-01|                  312|
| 103079215108|   2016-10-01|                  342|
| 936302870533|   2017-09-01|                  316|
+-------------+-------------+---------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.types import StringType

In [ ]:
left = expedia_with_months.filter(expedia_with_months["monthDiff"]>0).groupBy("hotel_id", "srch_ci_month").agg(count("id").alias("count_search_requests")).withColumnRenamed("srch_ci_month","srch_month")
right = expedia_with_months.filter(expedia_with_months["monthDiff"]>0).groupBy("hotel_id", "srch_co_month").agg(count("id").alias("count_search_requests")).withColumnRenamed("srch_co_month","srch_month")
left.union(right).groupBy("hotel_id", "srch_month").sum("count_search_requests").withColumnRenamed("sum(count_search_requests)","count_search_requests").show(10)

+-------------+----------+---------------------+
|     hotel_id|srch_month|count_search_requests|
+-------------+----------+---------------------+
|2130303778819|2017-09-01|                   56|
|2078764171268|2017-09-01|                   57|
|2980707303434|2017-08-01|                   26|
|2903397892096|2017-09-01|                   48|
|2869038153732|2017-09-01|                   53|
|1941325217799|2017-09-01|                   58|
|2817498546177|2016-10-01|                   20|
|1743756722184|2016-10-01|                   14|
|2705829396484|2016-10-01|                   26|
|2276332666881|2017-08-01|                   23|
+-------------+----------+---------------------+
only showing top 10 rows



In [ ]:
!ls -l

total 1300
drwxr-xr-x 2 root root    4096 Jan  1  1970 conf
drwxr-xr-x 3 root root    4096 Apr 18 12:20 eventlogs
drwxr-xr-x 2 root root    4096 Apr 18 13:45 ganglia
drwxr-xr-x 2 root root    4096 Apr 18 13:00 logs
drwxr-xr-x 5 root root    4096 Apr 18 12:25 metastore_db
-r-xr-xr-x 1 root root 1307454 Jan  1  1970 preload_class.lst
